In [1]:
import scipy.io.wavfile as wav
import scipy.signal as signal
from matplotlib import pyplot as plt
import os
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn.functional as F
import gc
from pathlib import Path

In [2]:
# Define the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
#device = 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [3]:
#chose test song
path = "genres_user_test/hiphop"
name = "PYN"

In [4]:
# Custom Dataset class
class NumpyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.Zsamples = []
        self.Tsamples = []
        self.Fsamples = []
        
        # Load file paths and labels
        for label, class_dir in enumerate(os.listdir(root_dir)):
            class_path = os.path.join(root_dir, class_dir)
            if os.path.isdir(class_path):
                if (class_path != path):
                    continue
                for file_name in os.listdir(class_path):
                    if file_name.endswith("Z.npy"):
                        file_path = os.path.join(class_path, file_name)
                        self.Zsamples.append((file_path, label))
                    elif file_name.endswith("T.npy"):
                        file_path = os.path.join(class_path, file_name)
                        self.Tsamples.append((file_path, label))
                    elif file_name.endswith("F.npy"):
                        file_path = os.path.join(class_path, file_name)
                        self.Fsamples.append((file_path, label))
    
    def __len__(self):
        return len(self.Zsamples)
    
    def __getitem__(self, idx):
        file_path, label = self.Zsamples[idx]
        data = np.load(file_path)  # Load numpy array
        real = np.real(data)
        imag = np.imag(data)
        mag = np.abs(data)
        angle = np.angle(data)
        real = np.expand_dims(real, axis=0)
        imag = np.expand_dims(imag, axis=0)
        mag = np.expand_dims(mag, axis=0)
        angle = np.expand_dims(angle, axis=0)
        data = np.concatenate((real, imag, mag, angle), axis=0)
        data = torch.tensor(data, dtype=torch.float32)  # Convert to PyTorch tensor
        
        if self.transform:
            data = self.transform(data)
        
        return data, label

    def spectrograms(self, idx):
        Zfile_path, label = self.Zsamples[idx]
        Tfile_path, label = self.Tsamples[idx]
        Ffile_path, label = self.Fsamples[idx]
        Zdata = np.load(Zfile_path)  # Load numpy array
        Tdata = np.load(Tfile_path)  # Load numpy array
        Fdata = np.load(Ffile_path)  # Load numpy array
        print(Zdata.shape)
        # Create a 2x2 subplot grid
        fig, axes = plt.subplots(5, 1, figsize=(10, 16))
        
        # First subplot
        c1 = axes[0].pcolormesh(Tdata, Fdata, np.log(np.abs(Zdata)), cmap='gnuplot')
        fig.colorbar(c1, ax=axes[0])
        axes[0].set_title("Spectrogram Magnitude")
        
        # Second subplot
        c2 = axes[1].pcolormesh(Tdata, Fdata, np.angle(Zdata), cmap='gnuplot')
        fig.colorbar(c2, ax=axes[1])
        axes[1].set_title("Spectrogram Angle")      

        # Third subplot
        c3 = axes[2].pcolormesh(Tdata, Fdata, np.log(np.square(np.real(Zdata))), cmap='gnuplot')
        fig.colorbar(c3, ax=axes[2])
        axes[2].set_title("Spectrogram Real")  

        # Fourth subplot
        c4 = axes[3].pcolormesh(Tdata, Fdata, np.log(np.square(np.imag(Zdata))), cmap='gnuplot')
        fig.colorbar(c4, ax=axes[3])
        axes[3].set_title("Spectrogram Imag")  

        # Fifth subplot
        c5 = axes[4].pcolormesh(Tdata, Fdata, np.log(np.square(np.imag(Zdata)) + np.square(np.real(Zdata))), cmap='gnuplot')
        fig.colorbar(c4, ax=axes[4])
        axes[4].set_title("Spectrogram Imag + Real") 

In [5]:
# Define the transformations
# - ToTensor: Converts the image to a PyTorch tensor
# - Normalize: Normalizes using mean and std of the ImageNet dataset
transform = transforms.Compose([
    transforms.CenterCrop((544, 240)),
    transforms.Normalize((0,), (0.5,))
])

In [6]:
# Define the model
class MusicNet(nn.Module):
    def __init__(self):
        super(MusicNet, self).__init__()
        self.cLayer1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding='same', groups=4),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.cLayer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same'),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.cLayer3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding='same'),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.cLayer4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fLayer = nn.Sequential(
            nn.Linear(in_features=512*34*15, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10)
        )

    def forward(self, x, return_features=False):
        # Apply the first convolution and activation function
        # print('x dims', x.shape)
        
        x = self.cLayer1(x)
        x = self.cLayer2(x)
        x = self.cLayer3(x)
        x = self.cLayer4(x)
        
        x = x.view(x.shape[0], 512*34*15)       # Flatten before passing to fully connected layers
        
        if not return_features:
            x = self.fLayer(x)

        return x

# Instantiate the model
model = MusicNet().to(device)

# Print the model architecture
print(model)

MusicNet(
  (cLayer1): Sequential(
    (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=4)
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cLayer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cLayer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runn

In [7]:
model_path = "MusicGenreClassifier.pth"
model = MusicNet().to(device)
model.load_state_dict(torch.load(model_path, weights_only=False))

<All keys matched successfully>

In [8]:
#before loading music, make sure it is at least 6 seconds long and sampled at 22050 Hz
# ffmpeg -i foo.mp3 -vn -acodec pcm_s16le -ac 1 -ar 22050 -f wav foo.wav
num_divisions = 5
clip_length = 132301

sample_rate, samples = wav.read(path + "/" + name + ".wav") # extract audio
        
# convert to mono
if len(samples.shape) > 1:
    # Do a mean of all channels and keep it in one channel
    samples = np.mean(samples, axis=1)

# pad so divisible by num_divisions
samples = np.append(samples, np.zeros(num_divisions - (len(samples) % num_divisions)))

# perform STFT on 6sec samples
clip_samples=[]
start = 0
end = start + clip_length

while (end < len(samples)):
    clip_samples.append(samples[start: end])
    start = end
    end = end + clip_length

i=0
for sample in clip_samples:
    SFT = signal.ShortTimeFFT.from_window(win_param='tukey', 
                                          fs=sample_rate, 
                                          nperseg=sample_rate//20,      #make 20Hz minimum sampled frequency
                                          noverlap=(sample_rate//20)//2,  #50% overlap
                                          fft_mode='onesided', 
                                          scale_to='magnitude', 
                                          phase_shift=None,
                                          symmetric_win=True)
    Zxx = SFT.stft(sample)
    t = SFT.t(len(sample))
    f = SFT.f
    np.save(path + "/" + name + "_" + str(i) + "_Z.npy", Zxx)
    np.save(path + "/" + name + "_" + str(i) + "_T.npy", t)
    np.save(path + "/" + name + "_" + str(i) + "_F.npy", f)
    i+=1
    
    

In [9]:
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

In [10]:
# Testing loop
def music_classifier(model, music_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    print("Real Genre : Guess")

    with torch.no_grad():
        for images, labels in music_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            # Print the real and predicted genre for each sample in the batch
            for label, pred in zip(labels, predicted):
                print(f"{genres[label]:10} : {genres[pred]:10}")
                
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            

    accuracy = 100 * correct / total
    print(f'Test Accuracy of the model on the test dataset: {accuracy:.2f}%')

In [11]:
root_dir = "genres_user_test/"
music_dataset = NumpyDataset(root_dir=root_dir, transform=transform)

print(len(music_dataset))

music_loader = DataLoader(dataset=music_dataset, batch_size=1, shuffle=False)

music_classifier(model, music_loader)

46
Real Genre : Guess
hiphop     : classical 
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : reggae    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hiphop    
hiphop     : hipho